In [91]:
# Databricks notebook source
from databricks.connect import DatabricksSession
from pyspark.sql.types import IntegerType, StringType, StructField, StructType, BooleanType, FloatType, DateType, TimestampType, ArrayType, MapType, StructType
from pyspark.sql.functions import explode, explode_outer, col
from pyspark.sql import Row
import pandas as pd
import datetime
#
spark = DatabricksSession.builder.profile("test").getOrCreate()

#### Create Spark DataFrame from different python structures

In [58]:
# create a multicolumn dataframe and set name of columns and types
users_list = [(1, 'Scott'), (2, 'Donald'), (3, 'Mickey'), (4, 'Elvis')]

spark.createDataFrame(users_list, 'user_id int, user_first_name string').show()


+-------+---------------+
|user_id|user_first_name|
+-------+---------------+
|      1|          Scott|
|      2|         Donald|
|      3|         Mickey|
|      4|          Elvis|
+-------+---------------+



In [6]:
# transform spark dataframe into list of Rows
df.collect()

[Row(user_id=1, user_first_name='Scott'),
 Row(user_id=2, user_first_name='Donald'),
 Row(user_id=3, user_first_name='Mickey'),
 Row(user_id=4, user_first_name='Elvis')]

In [57]:
Row(name="Alice", age=11)

Row(name='Alice', age=11)

In [19]:
# Convert list of list to list of Spark Rows
# use asterik to unpack the attributes of each user and pass them as arguments to the Row class
user_rows = [Row(*user) for user in users_list]

In [55]:
# convert list of rows to dataframe, SAME as creating a dataframe from a list of tuples
spark.createDataFrame(user_rows, 'user_id int, user_first_name string').show()

+-------+---------------+
|user_id|user_first_name|
+-------+---------------+
|      1|          Scott|
|      2|         Donald|
|      3|         Mickey|
|      4|          Elvis|
+-------+---------------+



In [54]:
# convert pandas dataframe to spark dataframe
spark.createDataFrame(pd.DataFrame(users_list), 'user_id int, user_first_name string').show()

+-------+---------------+
|user_id|user_first_name|
+-------+---------------+
|      1|          Scott|
|      2|         Donald|
|      3|         Mickey|
|      4|          Elvis|
+-------+---------------+



#### Data Types


In [100]:
users = [
    {
        "id": 1,
        "first_name": "Corrie",
        "last_name": "Van den Oord",
        "email": "cvandenoord0@etsy.com",
        "phone_numbers_array": ["+1 234 567 8901", "+1 234 567 8911"],
        "phone_numbers_map": {"mobile": "+1 234 567 8901", "home": "+1 234 567 8911"},
        "phone_numbers_struct": Row(mobile="+1 234 567 8901", home="+1 234 567 8911"),
        "is_customer": True,
        "amount_paid": 1000.55,
        "customer_from": datetime.date(2021, 1, 15),
        "last_updated_ts": datetime.datetime(2021, 2, 10, 1, 15, 0)
    },
    {
        "id": 2,
        "first_name": "Nikolaus",
        "last_name": "Brewitt",
        "email": "nbrewitt1@dailymail.co.uk",
        "phone_numbers_array": ["+1 234 567 8923", "+1 234 567 8934"],
        "phone_numbers_map": {"mobile": "+1 234 567 8923", "home": "+1 234 567 8934"},
        "phone_numbers_struct":  Row(mobile="+1 234 567 8923", home="1 234 567 8934"),
        "is_customer": True,
        "amount_paid": 900.0,
        "customer_from": datetime.date(2021, 2, 14),
        "last_updated_ts": datetime.datetime(2021, 2, 18, 3, 33, 0)
    },
    {
        "id": 3,
        "first_name": "Orelie",
        "last_name": "Penney",
        "email": "openney2@vistaprint.com",
        "phone_numbers_array": ["+1 714 512 9752", "+1 714 512 6601"],
        "phone_numbers_map": {"mobile": "+1 714 512 9752", "home": "+1 714 512 6601"},
        "phone_numbers_struct": Row(mobile="+1 714 512 9752", home="+1 714 512 6601"),
        "is_customer": True,
        "amount_paid": 850.55,
        "customer_from": datetime.date(2021, 1, 21),
        "last_updated_ts": datetime.datetime(2021, 3, 15, 15, 16, 55)
    },
    {
        "id": 4,
        "first_name": "Ashby",
        "last_name": "Maddocks",
        "email": "amaddocks3@home.pl",
        "phone_numbers_array": None,
        "phone_numbers_map": None,
        "phone_numbers_struct": Row(mobile=None, home=None),
        "is_customer": False,
        "amount_paid": None,
        "customer_from": None,
        "last_updated_ts": datetime.datetime(2021, 4, 10, 17, 45, 30)
    },
    {
        "id": 5,
        "first_name": "Kurt",
        "last_name": "Rome",
        "email": "krome4@shutterfly.com",
        "phone_numbers_array": ["+1 817 934 7142"],
        "phone_numbers_map": {"mobile": "+1 817 934 7142"},
        "phone_numbers_struct": Row(mobile="+1 817 934 7142", home=None),
        "is_customer": False,
        "amount_paid": None,
        "customer_from": None,
        "last_updated_ts": datetime.datetime(2021, 4, 2, 0, 55, 18)
    }
]


# you can use a string, a list of names for the columns or a list of Spark StructFields passed inside StructType
#users_schema = '''
#    id INT,
#    first_name STRING,
#    last_name STRING,
#    email STRING,
#    is_customer BOOLEAN,
#    amount_paid FLOAT,
#    customer_from DATE,
#    last_updated_ts TIMESTAMP
#'''

users_schema = StructType([
    StructField('id', IntegerType()),
    StructField('first_name', StringType()),
    StructField('last_name', StringType()),
    StructField('email', StringType()),
    
    # Custom type ArrayType
    StructField('phone_numbers_array', ArrayType(StringType())),

    # Custom type MapType
    StructField('phone_numbers_map', MapType(StringType(), StringType())),

    # Custom type StructType
    StructField('phone_numbers_struct', StructType([
                                            StructField("mobile", StringType()), 
                                            StructField("home", StringType())
                                        ])   
    ),


    StructField('is_customer', BooleanType()),
    StructField('amount_paid', FloatType()),
    StructField('customer_from', DateType()),
    StructField('last_updated_ts', TimestampType())
])

In [101]:
# Create Spark DataFrame from list of tuples and schema
schema_df = spark.createDataFrame(users)
schema_df.show(truncate=False)

+-----------+-------------+-------------------------+----------+---+-----------+------------+-------------------+----------------------------------+----------------------------------------------------+----------------------------------+
|amount_paid|customer_from|email                    |first_name|id |is_customer|last_name   |last_updated_ts    |phone_numbers_array               |phone_numbers_map                                   |phone_numbers_struct              |
+-----------+-------------+-------------------------+----------+---+-----------+------------+-------------------+----------------------------------+----------------------------------------------------+----------------------------------+
|1000.55    |2021-01-15   |cvandenoord0@etsy.com    |Corrie    |1  |true       |Van den Oord|2021-02-10 04:15:00|[+1 234 567 8901, +1 234 567 8911]|{mobile -> +1 234 567 8901, home -> +1 234 567 8911}|{+1 234 567 8901, +1 234 567 8911}|
|900.0      |2021-02-14   |nbrewitt1@dailymail.co.uk

In [102]:
schema_df.printSchema()

root
 |-- amount_paid: double (nullable = true)
 |-- customer_from: date (nullable = true)
 |-- email: string (nullable = true)
 |-- first_name: string (nullable = true)
 |-- id: long (nullable = true)
 |-- is_customer: boolean (nullable = true)
 |-- last_name: string (nullable = true)
 |-- last_updated_ts: timestamp (nullable = true)
 |-- phone_numbers_array: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- phone_numbers_map: map (nullable = true)
 |    |-- key: string
 |    |-- value: string (valueContainsNull = true)
 |-- phone_numbers_struct: struct (nullable = true)
 |    |-- mobile: string (nullable = true)
 |    |-- home: string (nullable = true)



##### Special types

- ARRAY
Can come from python data type list, can only have one type

- STRUCT
requires a few api functions to convert from python data types, scan have multiple types

- MAP
Can come from python data type dictionary

In [105]:
# explode <- brings out a row for each element in a special type (only returns rows with non-null values) explode_outer (returns all rows)
# array
schema_df. \
    withColumn('phone_number_array', explode_outer('phone_numbers_array')). \
    drop('phone_numbers_array', 'phone_numbers_map', 'phone_numbers_struct'). \
    show()

+-----------+-------------+--------------------+----------+---+-----------+------------+-------------------+------------------+
|amount_paid|customer_from|               email|first_name| id|is_customer|   last_name|    last_updated_ts|phone_number_array|
+-----------+-------------+--------------------+----------+---+-----------+------------+-------------------+------------------+
|    1000.55|   2021-01-15|cvandenoord0@etsy...|    Corrie|  1|       true|Van den Oord|2021-02-10 04:15:00|   +1 234 567 8901|
|    1000.55|   2021-01-15|cvandenoord0@etsy...|    Corrie|  1|       true|Van den Oord|2021-02-10 04:15:00|   +1 234 567 8911|
|      900.0|   2021-02-14|nbrewitt1@dailyma...|  Nikolaus|  2|       true|     Brewitt|2021-02-18 06:33:00|   +1 234 567 8923|
|      900.0|   2021-02-14|nbrewitt1@dailyma...|  Nikolaus|  2|       true|     Brewitt|2021-02-18 06:33:00|   +1 234 567 8934|
|     850.55|   2021-01-21|openney2@vistapri...|    Orelie|  3|       true|      Penney|2021-03-15 18:16

In [117]:
#map
schema_df.select('*', explode('phone_numbers_map')). \
    withColumnRenamed('key', 'phone_type'). \
    withColumnRenamed('value', 'phone_number'). \
    drop('phone_numbers_array', 'phone_numbers_map', 'phone_numbers_struct'). \
    show()

+-----------+-------------+--------------------+----------+---+-----------+------------+-------------------+----------+---------------+
|amount_paid|customer_from|               email|first_name| id|is_customer|   last_name|    last_updated_ts|phone_type|   phone_number|
+-----------+-------------+--------------------+----------+---+-----------+------------+-------------------+----------+---------------+
|    1000.55|   2021-01-15|cvandenoord0@etsy...|    Corrie|  1|       true|Van den Oord|2021-02-10 04:15:00|    mobile|+1 234 567 8901|
|    1000.55|   2021-01-15|cvandenoord0@etsy...|    Corrie|  1|       true|Van den Oord|2021-02-10 04:15:00|      home|+1 234 567 8911|
|      900.0|   2021-02-14|nbrewitt1@dailyma...|  Nikolaus|  2|       true|     Brewitt|2021-02-18 06:33:00|    mobile|+1 234 567 8923|
|      900.0|   2021-02-14|nbrewitt1@dailyma...|  Nikolaus|  2|       true|     Brewitt|2021-02-18 06:33:00|      home|+1 234 567 8934|
|     850.55|   2021-01-21|openney2@vistapri...|

In [131]:
#struct
schema_df.select('*', "phone_numbers_struct.*").show()

+---------------+---------------+-----------+-------------+--------------------+----------+---+-----------+------------+-------------------+--------------------+--------------------+--------------------+
|         mobile|           home|amount_paid|customer_from|               email|first_name| id|is_customer|   last_name|    last_updated_ts| phone_numbers_array|   phone_numbers_map|phone_numbers_struct|
+---------------+---------------+-----------+-------------+--------------------+----------+---+-----------+------------+-------------------+--------------------+--------------------+--------------------+
|+1 234 567 8901|+1 234 567 8911|    1000.55|   2021-01-15|cvandenoord0@etsy...|    Corrie|  1|       true|Van den Oord|2021-02-10 04:15:00|[+1 234 567 8901,...|{mobile -> +1 234...|{+1 234 567 8901,...|
|+1 234 567 8923| 1 234 567 8934|      900.0|   2021-02-14|nbrewitt1@dailyma...|  Nikolaus|  2|       true|     Brewitt|2021-02-18 06:33:00|[+1 234 567 8923,...|{mobile -> +1 234...|{+

In [106]:
# array
schema_df. \
    select('id', col('phone_numbers_array')[0].alias('mobile'), col('phone_numbers_array')[1].alias('home')). \
    show()

+---+---------------+---------------+
| id|         mobile|           home|
+---+---------------+---------------+
|  1|+1 234 567 8901|+1 234 567 8911|
|  2|+1 234 567 8923|+1 234 567 8934|
|  3|+1 714 512 9752|+1 714 512 6601|
|  4|           NULL|           NULL|
|  5|+1 817 934 7142|           NULL|
+---+---------------+---------------+



In [109]:
# map
schema_df. \
    select('id', col('phone_numbers_map')['mobile'].alias('mobile'), col('phone_numbers_map')['home'].alias('home')). \
    show()

+---+---------------+---------------+
| id|         mobile|           home|
+---+---------------+---------------+
|  1|+1 234 567 8901|+1 234 567 8911|
|  2|+1 234 567 8923|+1 234 567 8934|
|  3|+1 714 512 9752|+1 714 512 6601|
|  4|           NULL|           NULL|
|  5|+1 817 934 7142|           NULL|
+---+---------------+---------------+



In [110]:
# struct
schema_df. \
    select('id', col('phone_numbers_struct')['mobile'].alias('mobile'), col('phone_numbers_struct')['home'].alias('home')). \
    show()

+---+---------------+---------------+
| id|         mobile|           home|
+---+---------------+---------------+
|  1|+1 234 567 8901|+1 234 567 8911|
|  2|+1 234 567 8923| 1 234 567 8934|
|  3|+1 714 512 9752|+1 714 512 6601|
|  4|           NULL|           NULL|
|  5|+1 817 934 7142|           NULL|
+---+---------------+---------------+

